### Your name:

<pre> MAYURA MURALIBABU</pre>

### Collaborators:

<pre> KUSHAL TOPNANI </pre>


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Markdown referece can be found here:
    http://nestacms.com/docs/creating-content/markdown-cheat-sheet

**Q1. Why would it be a problem if our training set and test set are the same.**

<pre>If training set and test is same then the we will get a 100% accuracy rate which is pratically impossible. Also, model will not be able to learn for a generalise data</pre>


Open the housing data


In [2]:
import os
import tarfile
from six.moves import urllib
import pandas as pd


DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
fetch_housing_data()


def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Build full pipeline for the data analysis following the example of the notebook we went through in class.
 Hint: the main part requested to change is the algorithm used (Lasso regression)

If you want to learn more about the Lasso regression, see resources below:
- http://scikit-learn.org/stable/modules/linear_model.html#lasso
- https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/

#### Considerations for building pipeline:

- Split data into training and testing sets below.
- Convert all categorical data to one-hot vectors below
- Normalize all non-categorical data 
-  Perform Lasso-based regression using a variety of values for $\alpha$ between 0 and 1 via a grid search where  *housing_labels* is the output and all other features are the input (similar to as seen in lecture two.)

In [3]:
import re
import scipy
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import Lasso

pd.options.display.max_columns = None
np.random.seed(0)


df = pd.read_csv('datasets/housing/housing.csv')

df=df.copy()

def rmse(a, b):
    return np.sqrt(np.mean((a-b)**2))

In [4]:
df.sample(10, random_state=0)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
14740,-117.05,32.58,22.0,2101.0,399.0,1551.0,371.0,4.1518,136900.0,NEAR OCEAN
10101,-117.97,33.92,32.0,2620.0,398.0,1296.0,429.0,5.7796,241300.0,<1H OCEAN
20566,-121.84,38.65,29.0,3167.0,548.0,1554.0,534.0,4.3487,200700.0,INLAND
2670,-115.60,33.20,37.0,709.0,187.0,390.0,142.0,2.4511,72500.0,INLAND
15709,-122.43,37.79,25.0,1637.0,394.0,649.0,379.0,5.0049,460000.0,NEAR BAY
439,-122.29,37.85,52.0,477.0,119.0,218.0,106.0,2.5682,120000.0,NEAR BAY
845,-122.07,37.58,16.0,1606.0,240.0,1117.0,268.0,6.0661,247000.0,NEAR BAY
3768,-118.43,34.17,37.0,1982.0,331.0,794.0,340.0,5.9275,336900.0,<1H OCEAN
964,-121.89,37.68,12.0,7490.0,1207.0,3329.0,1136.0,6.3373,339700.0,<1H OCEAN
8681,-118.33,33.87,44.0,724.0,133.0,373.0,133.0,3.9167,265600.0,<1H OCEAN


In [5]:
[(x, y) for x, y in zip(df.isna().sum(), df.isna().sum().index) if x > 0]

[(207, 'total_bedrooms')]

In [6]:
print(housing.info)

<bound method DataFrame.info of        longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0        -122.23     37.88                41.0        880.0           129.0   
1        -122.22     37.86                21.0       7099.0          1106.0   
2        -122.24     37.85                52.0       1467.0           190.0   
3        -122.25     37.85                52.0       1274.0           235.0   
4        -122.25     37.85                52.0       1627.0           280.0   
...          ...       ...                 ...          ...             ...   
20635    -121.09     39.48                25.0       1665.0           374.0   
20636    -121.21     39.49                18.0        697.0           150.0   
20637    -121.22     39.43                17.0       2254.0           485.0   
20638    -121.32     39.43                18.0       1860.0           409.0   
20639    -121.24     39.37                16.0       2785.0           616.0   

       population  

In [7]:
##IMPUTERS ##

print(df[['total_bedrooms']].values[:, 0])
tfmr = SimpleImputer(strategy='median')
print(tfmr.fit_transform(df[['total_bedrooms']].values)[:, 0])

print(df[['ocean_proximity']].values[:, 0])
tfmr = SimpleImputer(strategy='constant', fill_value='Other')

print(tfmr.fit_transform(df[['ocean_proximity']].values)[:, 0])

[ 129. 1106.  190. ...  485.  409.  616.]
[ 129. 1106.  190. ...  485.  409.  616.]
['NEAR BAY' 'NEAR BAY' 'NEAR BAY' ... 'INLAND' 'INLAND' 'INLAND']
['NEAR BAY' 'NEAR BAY' 'NEAR BAY' ... 'INLAND' 'INLAND' 'INLAND']


In [8]:

##Convert all categorical data to one-hot vectors below##

print(df[['ocean_proximity']].values)
tfmr = OneHotEncoder(sparse=False)
print(tfmr.fit_transform(df[['ocean_proximity']].values))

[['NEAR BAY']
 ['NEAR BAY']
 ['NEAR BAY']
 ...
 ['INLAND']
 ['INLAND']
 ['INLAND']]
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


In [9]:
#variable buckets:
dependent_vars=['median_house_value']
categorical_vars = ['ocean_proximity']
discrete_vars =['total_rooms','total_bedrooms','housing_median_age']
continuous_vars=['longitude','latitude','population','households','median_income']

In [10]:

##Normalize all non-categorical data##

print(df[['housing_median_age']].values[:, 0])
tfmr = MinMaxScaler()
print(tfmr.fit_transform(df[['housing_median_age']].values)[:, 0])

print(df[['total_rooms']].values[:, 0])
tfmr = MinMaxScaler()
print(tfmr.fit_transform(df[['total_rooms']].values)[:, 0])

print(df[['median_income']].values[:, 0])
tfmr = MinMaxScaler()
print(tfmr.fit_transform(df[['median_income']].values)[:, 0])

print(df[['median_house_value']].values[:, 0])
tfmr = MinMaxScaler()
print(tfmr.fit_transform(df[['median_house_value']].values)[:, 0])

print(df[['population']].values[:, 0])
tfmr = MinMaxScaler()
print(tfmr.fit_transform(df[['population']].values)[:, 0])

print(df[['total_bedrooms']].values[:, 0])
tfmr = MinMaxScaler()
print(tfmr.fit_transform(df[['total_bedrooms']].values)[:, 0])

print(df[['households']].values[:, 0])
tfmr = MinMaxScaler()
print(tfmr.fit_transform(df[['households']].values)[:, 0])

print(df[['latitude']].values[:, 0])
tfmr = MinMaxScaler()
print(tfmr.fit_transform(df[['latitude']].values)[:, 0])

print(df[['longitude']].values[:, 0])
tfmr =  MinMaxScaler()
print(tfmr.fit_transform(df[['longitude']].values)[:, 0])


[41. 21. 52. ... 17. 18. 16.]
[0.78431373 0.39215686 1.         ... 0.31372549 0.33333333 0.29411765]
[ 880. 7099. 1467. ... 2254. 1860. 2785.]
[0.02233074 0.18050257 0.03726029 ... 0.05727657 0.04725571 0.07078183]
[8.3252 8.3014 7.2574 ... 1.7    1.8672 2.3886]
[0.53966842 0.53802706 0.46602805 ... 0.08276438 0.09429525 0.13025338]
[452600. 358500. 352100. ...  92300.  84700.  89400.]
[0.90226638 0.70824656 0.69505074 ... 0.15938285 0.14371281 0.15340349]
[ 322. 2401.  496. ... 1007.  741. 1387.]
[0.00894083 0.0672104  0.01381765 ... 0.0281398  0.02068444 0.03879032]
[ 129. 1106.  190. ...  485.  409.  616.]
[0.01986344 0.17147734 0.02932961 ... 0.07510863 0.06331471 0.09543762]
[ 126. 1138.  177. ...  433.  349.  530.]
[0.02055583 0.18697583 0.02894261 ... 0.07104095 0.05722743 0.08699227]
[37.88 37.86 37.85 ... 39.43 39.43 39.37]
[0.5674814  0.565356   0.5642933  ... 0.73219979 0.73219979 0.72582359]
[-122.23 -122.22 -122.24 ... -121.22 -121.32 -121.24]
[0.21115538 0.21215139 0.210

In [11]:
# Categorical - impute, one hot encode
cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value='Other'))
cat_ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
cat_steps = [cat_si_step, cat_ohe_step]
cat_pipe = Pipeline(cat_steps)
cat_transformers = [('cat', cat_pipe, categorical_vars)]

# Numerical - impute, scale
num_si_step = ('si', SimpleImputer(strategy='median'))
num_scl_step = ('scl', MinMaxScaler())
num_steps = [num_si_step, num_scl_step]
num_pipe = Pipeline(num_steps)
num_transformers = [('num', num_pipe, discrete_vars + continuous_vars)]




In [12]:

ct = ColumnTransformer(transformers=cat_transformers+ num_transformers)
ct.fit(df[categorical_vars + discrete_vars + continuous_vars])
X = ct.transform(df[categorical_vars + discrete_vars + continuous_vars])

y = df['median_house_value'].values

In [13]:

## split train and test##
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16512, 13) (4128, 13) (16512,) (4128,)


In [14]:
##Performing Lasso Regression##
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

ls = Lasso(normalize=False, max_iter=100000,tol=1.0)


In [15]:
## various alpha values -- Lasso Regression## 
param_grid = [{'alpha':[1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]}]
housing_grid = GridSearchCV(ls, param_grid,cv=5)

In [16]:
## fit for Lasso regression ## 
housing_grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=100000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=1.0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01, 1,
                                    5, 10, 20]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [17]:
housing_grid.best_score_

0.614136062817232

In [18]:
housing_grid.cv_results_

{'mean_fit_time': array([0.00859623, 0.007197  , 0.00619736, 0.0061974 , 0.00539827,
        0.0055984 , 0.00519814, 0.00539837, 0.00439763, 0.00439816]),
 'std_fit_time': array([0.00272572, 0.00039907, 0.00097954, 0.00097899, 0.00048998,
        0.00048996, 0.00039988, 0.0004898 , 0.0004892 , 0.00048994]),
 'mean_score_time': array([0.00139871, 0.00159874, 0.00139985, 0.00099945, 0.00099945,
        0.00099907, 0.00139933, 0.00119872, 0.00080042, 0.00099988]),
 'std_score_time': array([4.89181163e-04, 4.89804534e-04, 4.89318912e-04, 3.01578299e-07,
        9.29526743e-07, 7.16843432e-07, 4.90037648e-04, 3.99375001e-04,
        4.00209555e-04, 6.64157308e-07]),
 'param_alpha': masked_array(data=[1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01, 1, 5, 10, 20],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 1e-15},
  {'alpha': 1e-10},
  {'alpha': 1e-08},
  {'a

In [19]:
train_score = housing_grid.score(X_train,y_train)
test_score = housing_grid.score(X_test,y_test)
print(train_score)
print(test_score)

0.614631826702569
0.600652762781732


In [20]:
print(housing_grid.best_estimator_)

Lasso(alpha=1e-15, copy_X=True, fit_intercept=True, max_iter=100000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=1.0, warm_start=False)


In [21]:
## RMSE ---- Lasso Regression
y_pred=housing_grid.predict(X_train)
print(y_pred)
print(rmse(y_train,y_pred))

[134097.97339691 279490.08910499 223764.53820632 ... 225291.19319365
 296437.46372701  36379.73122094]
71816.83509474852


In [23]:
value = ls.fit(X_train,y_train)
print(value)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=100000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=1.0, warm_start=False)


In [24]:
print(value.coef_)

[  22321.78239726  -54516.67291509  222372.41411428   36007.01660479
   40542.36930566  274630.2966332  -122900.34052013   45708.08552147
  -25168.07097366  -21501.0346222  -422980.78668261  231958.3886294
  521226.90640408]


In [25]:
##Linear Regression##
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, np.ravel(y_train))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [26]:
def display_results(model, X, y):
    print("RMSE:", rmse(model.predict(X), y))
    print("Predicted 1-5:", model.predict(X_test[0:5]))
    print("Actual 1-5:", y_test[0:5])
    
display_results(reg, X, y)

RMSE: 68756.98553551809
Predicted 1-5: [213632. 290496. 177664.  84800. 288128.]
Actual 1-5: [136900. 241300. 200700.  72500. 460000.]


**Q2. Why is it necessary to normalize all continuous variables before performing Lasso?**

<pre>Normalizing data will bring all the numeric values to a common scale and give a better prediction for a global minimum. When normalizing the parameters,the predictors must not outweight one another.  </pre>

### Conclusions
**Q3. For what values of $\alpha$ does Lasso perform best? Does it perform as well on the housing data as the linear regressor from the lectures? Why do you think this is?**

<pre> We have tried the Lasso Regression for alpha values:
1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20
Train_score = 0.614631826702569
Test_score= 0.600652762781732
rmse=71816.8350947485

At alpha = 1 the Lasso perform best. It performs as well on the datasets for linear regression.This  is because we have normalized the numerical variables so that they perform well together. Also, since alpha = 1 performs best, this means that the coefficients are that of a simple linear regression analysis, producing similar values.
</pre>

### Submit your notebook

Submit your solution on Canvas